In [3]:
import json
import csv
import pandas as pd
import sqlite3
import networkx as nx

In [2]:
ls

Collection JSONs/                  nft_database.db
all_transactions.csv               nft_database_simple.db
generate_top_pr_edges.sql          nft_database_simple_new.db
json_to_sqlite(not working).ipynb  page_rank_files/
json_to_sqlite(working).ipynb      terraform.csv
network analysis.ipynb             union_tables.sql
network_collection_pipe.ipynb


In [11]:
conn = sqlite3.connect('nft_database_simple_new.db')
c = conn.cursor()
collections = ['Meebits', 'Terraform', 'Loot', 'dotdotdots', 'CloneX', 'Cool Cats',
               'Azuki', 'Cryptopunks', 'Doodles', 'Bored Ape Yacht Club', 'Mutant Ape Yacht Club',
               'Bored Ape Kennel Club', 'World of Women', 'CrypToadz']

In [12]:
for collection_name in collections:
    collection = collection_name
    collectionWith = '"' + collection + '"'
    weight = 'weight'
    df_all_transactions = pd.read_sql_query('select from_address, to_address, ' + weight + ' from all_transactions_weighted where collection_name = ' + collectionWith, conn)
    df_edge = df_all_transactions[['from_address', 'to_address', weight]]
#     df_edge.head()
    query_wallets = 'select from_address from all_transactions where collection_name = ' + collectionWith + ' union select to_address from all_transactions where collection_name = ' + collectionWith
    df_wallets = pd.read_sql_query(query_wallets, conn)
#     df_wallets.head()
    G = nx.from_pandas_edgelist(df=df_edge, source='from_address', target='to_address', edge_attr=[weight])
    G.add_nodes_from(nodes_for_adding=df_wallets.from_address.tolist())
    df_edges = pd.DataFrame(G.edges.data())
    df_edges = df_edges.rename(columns={0: "source", 1: "target", 2: "attr"})
#     df_edges.head(15)
    pr = nx.pagerank(G, weight=weight)
    list = [(k, v) for k, v in pr.items()]
    df_pr = pd.DataFrame(list).rename(columns={0:'wallet_hash', 1:'page_rank'})
    df_pr = df_pr.sort_values(by=['page_rank'], ascending=False)
#     df_pr.head(50)
    cc = nx.clustering(G)
    df_cc = pd.DataFrame([(k, v) for k, v in cc.items()]).rename(columns={0:'wallet_hash', 1:'clustering coefficient'})
    df_cc = df_cc.sort_values(by=['clustering coefficient'], ascending=False)
#     df_cc
    df_pr.to_sql("page_rank_" + collection, conn, if_exists='replace', index=False)
    query = "select from_address as target, to_address as source, count(*) as value from \
        (select from_address, to_address from (select * from [page_rank_" + collection + "] limit 100) as a \
        join (select * from all_transactions where collection_name = " + collectionWith + ") as b on a.wallet_hash = b.from_address) as c \
        join (select * from [page_rank_" + collection + "] limit 100) as d \
        on c.to_address = d.wallet_hash \
        group by from_address, to_address \
        order by value DESC";
    df_pr_top_100_edge = pd.read_sql_query(query, conn)
    df_pr_top_100_edge.head()
    df_pr_top_100_edge.to_csv('./page_rank_files/pr_top_100_' + collection + '_edge.csv', index=False)
    df_pr_top_100 = pd.read_sql_query("SELECT a.wallet_hash as wallet_hash, page_rank, weighted_cluster_coeff from (SELECT * from [page_rank_" + collection + "] limit 100) as a JOIN all_wallets as b where a.wallet_hash = b.wallet_hash", conn)
#     df_pr_top_100.head()
    df_pr_top_100.to_csv("./page_rank_files/pr_top_100_" + collection + ".csv", index=False)